In [1]:
import pandas as pd 
import librosa
import os
import numpy as np
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:



meta = pd.read_csv('meta/esc50.csv')
file = meta.iloc[0]['filename']
base_folder = 'audio'
csv_folder = 'csvs'

In [3]:
search = os.path.join(csv_folder, "*.wav")
all_csv = glob.glob(search)
print(len(all_csv), all_csv[0])
arr = loadtxt(all_csv[0])



2000 csvs/1-100032-A-0.wav


In [12]:
# how to decide which to take to train and which to test?

esc10 = meta[meta['esc10'] == True]
X = esc10['filename']
y = esc10['category']

# add label encoder

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

le = preprocessing.LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)



train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)
train.groupby('category').count()
test.groupby('category').count()

,filename
category,
chainsaw,10
clock_tick,10
crackling_fire,10
crying_baby,10
dog,10
helicopter,10
rain,10
rooster,10
sea_waves,10


In [13]:
# for each file in training we need to get it's features 
X_train_vectors = []
for index, filename in X_train.items():
    csv_file_path = os.path.join(csv_folder, filename)
    features = loadtxt(csv_file_path, delimiter=',')
    X_train_vectors.append(features)

print(len(X_train))
X_test_vectors = []
for index, filename in X_test.items():
    csv_file_path = os.path.join(csv_folder, filename)
    features = loadtxt(csv_file_path, delimiter=',')
    X_test_vectors.append(features)

print(len(X_test_vectors))

300
100


In [14]:
# we can use feature scaling, but later 
classifier = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=0)
classifier.fit(X_train_vectors, y_train_encoded)
y_pred = classifier.predict(X_test_vectors)

In [18]:


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_encoded, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test_encoded, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_encoded, y_pred)))
print('Accuracy: ', accuracy_score(y_test_encoded, y_pred))
print(classification_report(y_test_encoded, y_pred, target_names=test.category.unique()))

Mean Absolute Error: 0.63
Mean Squared Error: 2.89
Root Mean Squared Error: 1.7
Accuracy:  0.77
                precision    recall  f1-score   support

      sneezing       0.89      0.80      0.84        10
           dog       0.57      0.40      0.47        10
     sea_waves       0.80      0.80      0.80        10
    clock_tick       0.82      0.90      0.86        10
    helicopter       0.73      0.80      0.76        10
crackling_fire       0.75      0.60      0.67        10
          rain       0.58      0.70      0.64        10
   crying_baby       1.00      0.90      0.95        10
      chainsaw       0.82      0.90      0.86        10
       rooster       0.75      0.90      0.82        10

      accuracy                           0.77       100
     macro avg       0.77      0.77      0.77       100
  weighted avg       0.77      0.77      0.77       100



In [13]:
# how to decide which to take to train and which to test?

esc10 = meta[meta['esc10'] == True]
X = esc10['filename']
y = esc10['category']

# add label encoder

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

le = preprocessing.LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)



train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)
print(train.groupby('category').count())
print(test.groupby('category').count())

# for each file in training we need to get it's features 
X_train_vectors = []
for index, filename in X_train.items():
    csv_file_path = os.path.join(csv_folder, filename)
    features = loadtxt(csv_file_path, delimiter=',')
    X_train_vectors.append(features)

print(len(X_train))
X_test_vectors = []
for index, filename in X_test.items():
    csv_file_path = os.path.join(csv_folder, filename)
    features = loadtxt(csv_file_path, delimiter=',')
    X_test_vectors.append(features)

print(len(X_test_vectors))

# we can use feature scaling, but later 
classifier = RandomForestClassifier(n_estimators=500, random_state=0)
classifier.fit(X_train_vectors, y_train_encoded)
y_pred = classifier.predict(X_test_vectors)


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_encoded, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test_encoded, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_encoded, y_pred)))
print('Accuracy: ', accuracy_score(y_test_encoded, y_pred))
print(classification_report(y_test_encoded, y_pred, target_names=test.category.unique()))


                filename
category                
chainsaw              30
clock_tick            30
crackling_fire        30
crying_baby           30
dog                   30
helicopter            30
rain                  30
rooster               30
sea_waves             30
sneezing              30
                filename
category                
chainsaw              10
clock_tick            10
crackling_fire        10
crying_baby           10
dog                   10
helicopter            10
rain                  10
rooster               10
sea_waves             10
sneezing              10
300
100
Mean Absolute Error: 0.63
Mean Squared Error: 2.75
Root Mean Squared Error: 1.6583123951777
Accuracy:  0.77
                precision    recall  f1-score   support

      sneezing       0.86      0.60      0.71        10
           dog       0.67      0.40      0.50        10
     sea_waves       0.82      0.90      0.86        10
    clock_tick       0.75      0.90      0.82        10
   